In [1]:
#training data: tiny shakespeare
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

#

--2024-10-04 06:32:18--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-10-04 06:32:19 (17.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



## Reading the input text

In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(f"length of dataset in characters: {len(text)} ")

length of dataset in characters: 1115394 


In [4]:
# Looking at the first 1500 characters
print(text[:1500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



## Extracting all unique characters that occur in the text

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## Implementing the tokenizer

In [ ]:
# Need a strategy to tokenize the text
# Tokenize: implies converting the raw text
# as a string to some sequence of integers
# according to some vocabulary of some possible elements

# since in this case we are building a character level language model
# so we are simply going to be translating individual characters into integers


In [6]:
# creating a mapping from characters to integers (AKA Tokenizing)
# we are building both the encoder and the decoder
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

# encoder: takes a string, outputs a list of integers
encode = lambda s: [stoi[c] for c in s]

# decoder: takes a list of integers, outputs a string
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello there"))
print(decode(encode("hello there")))

# The above is only one of many possible encoding/tokenizers
# e.g. SentencePiece (sub word tokenizers - usually adapted in practice)
# e.g. tiktoken (used by open AI)
# the one we presented is a character level tokenizer

[46, 43, 50, 50, 53, 1, 58, 46, 43, 56, 43]
hello there


### Encoding the entire text dataset and storing it in a torch.Tensor

In [7]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

# Printing the 1500 characters we saw earlier
# Will look as follows to the GPT
print(data[:1500])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56,  ..., 58, 53,  1])


## Train Validation Split of the dataset

In [8]:
# Hold 90% data for training
# 10% will be validation set
n = int(0.9*len(data))

train_data = data[:n]
val_data = data[n:]

In [9]:
# When we train the transformers we train on chunks of dataset
# These chunks are randomly selected
# These chunks are of a maximum length
# The max length is typically called block_size/context_length etc.
# Here we set the block_size and observe that block of the train_data

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
# The reason for adding the 1 is that
# by doing so we actually are saving the context
# after sequence of 18 likely it is 47 (1)
# after sequence of 18, 47 likely it is 56 (2)
# after sequence of 18, 47, 56, likely it is 57 (3)
# ...
# after sequence of 18, 47, 56, 57, 58, 1, 15, 47 likely it is 58 (8)

# code to show that
x = train_data[:block_size]   # input to transformer
y = train_data[1:block_size+1] # will be next character (targets) so offset by 1

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
# We train on all the eight examples above
# with context between 1 all the way up to context of block_size
# We train on it not just for computational reasons
# It is also done to make the transformer network used to seeing contexts
# all the way from as little as one to an entire block_size
# so we can start the sampling generation with as little as one character

In [12]:
# So far we have taken care of the time dimension
# Now we look towars batch dimension
# As we are sampling these chunks of text
# and feed them into a Transformer we're
# going to have many batches of multiple
# chunks of text all stacked up in a single tensor
# done mainly for efficiency

# As such we will generalize it and add a batch dimension to the above code
torch.manual_seed(1337)
batch_size = 4
block_size = 8


# function to create batch
def get_batch(split):
  # generate a small batch of data of input x and targets y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))

  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y


xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('------------')

for b in range(batch_size): # batch dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")


inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is